In [1]:
from typing import Optional, Callable, List, Union
import os
import itertools
import torch
import torch.nn as nn
from tqdm import tqdm
import wandb
from omegaconf import OmegaConf
import logging
import multiprocessing
from dotenv import load_dotenv
import yaml
from functools import partial
import time
from datetime import datetime
import lm_eval
from lm_eval.models.huggingface import HFLM
import numpy as np

# HF imports
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, LoraConfig, get_peft_model

from wmdp.rmu.utils import get_data, get_params, forward_with_cache
from latent_at.lat_methods import (
    projected_gradient_descent
)
from latent_at.utils import (
    get_minibatch,
    zero_nan_grads,
)
from latent_at.laa import (
    CustomHook,
    clear_hooks,
)
from latent_at.lat_helpers import (
    do_defense_step,
)
from latent_at.lat_datasets import WMDPLATDataCollator, WMDPLATTargetedDataCollator
from tasks.harmbench.FastHarmBenchEvals import run_general_evals
from tasks.general_capabilities.multiple_choice_tasks import WMDPTask, MMLUTask
from tasks.wmdp.WMDP_MCTask import WMDP_MCTask

load_dotenv()
hf_api_key = os.getenv("HF_API_KEY")

multiprocessing.set_start_method("spawn", force=True)


model_dict = {
    "LLAMA_7B": "meta-llama/Llama-2-7b-chat-hf",
    "LLAMA_34B": "meta-llama/Llama-2-34b-chat-hf",
    "ZEPHYR_7B": "HuggingFaceH4/zephyr-7b-beta",
    "ZEPHYR_34B": "HuggingFaceH4/zephyr-34b-beta",
}


class RunLAT:
    def __init__(
        self, 
        config: OmegaConf, 
        cuda_id: int,
        sweep_params: Optional[list[str]],
        init_callback: Optional[Callable] = None,
        post_def_callback: Optional[Callable] = None,
        post_adv_callback: Optional[Callable] = None,
    ):
        """
        Args:
            cuda_id: Each Run instance associated with a single GPU. Allows sweeps to call multiple Run instances on multiple GPUs.
            sweep_params: Any parameters that are changed by the sweep.
            init_callback: Typically, an evaluation function.
            post_def_callback: "".
            post_adv_callback: "".
        """
        if torch.cuda.is_available():
            self.device = torch.device(f"cuda:{cuda_id}")
        elif torch.backends.mps.is_available():
            self.device = torch.device("mps")
        else:
            self.device = torch.device("cpu")
        
        self.config = config
        self.rmu_config = config.rmu_config

        run = self._set_logger()
        if sweep_params: # For wandb sweeps: update with wandb values
            config = update_with_wandb_config(config, sweep_params)
        self.run_name = f"pgdlayers-{config.pgd_config.pgd_layers}_epsilon-{config.pgd_config.epsilon}_deflr-{config.def_config.outer_learning_rate}_pgdlr-{config.pgd_config.inner_learning_rate}_defsft-{config.def_config.def_loss_coefs.sft:.2f}_deftowards-{config.def_config.def_loss_coefs.toward:.2f}_defaway-{config.def_config.def_loss_coefs.away:.2f}_pgdtowards-{config.pgd_config.adv_loss_coefs.toward:.2f}_pgdaway-{config.pgd_config.adv_loss_coefs.away:.2f}_untargeted-{config.use_untargeted}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
        run.name = self.run_name

        # Model
        self.model, self.tokenizer = self._load_model(hf_api_key)

        ## Dataloaders
        if config.use_untargeted: # Untargeted LAT, use unlearn for defence and adversary
            forget_list = self._load_untargeted_wmdp_data()
            self.bio_dataloader, self.cyber_dataloader = self._make_untargeted_wmdp_dataloaders(forget_list, self.tokenizer, sft=False)
        else: # Use WMDP retain and unlearn - note logic is *totally different* to above
            assert config.forget_corpora == 'bio-forget-corpus,cyber-forget-corpus'
            bio_dataset, cyber_dataset = self._load_targeted_wmdp_data()
            self.bio_dataloader, self.cyber_dataloader = self._make_targeted_wmdp_dataloaders(bio_dataset, cyber_dataset, self.tokenizer)
        print(f"Dataloader length {len(self.bio_dataloader)} {len(self.cyber_dataloader)}")
        self.bio_dataloader, self.cyber_dataloader = itertools.cycle(self.bio_dataloader), itertools.cycle(self.cyber_dataloader)
        # SFT dataloader
        if config.def_config.def_loss_coefs.sft > 0:
            sft_dataset: list[str] = self._load_sft_dataset()
            self.sft_dataloader = self._make_untargeted_wmdp_dataloaders(sft_dataset, self.tokenizer, sft=True)
            self.sft_dataloader = itertools.cycle(self.sft_dataloader)
        else:
            assert config.def_config.def_loss_coefs["sft"] == 0
            self.sft_dataloader = None

        # Callbacks
        self.init_callback = init_callback
        self.post_adv_callback = post_adv_callback
        self.post_def_callback = post_def_callback
        
        if config.only_train_lora is None:
            self.only_train_lora = isinstance(self.model, PeftModel)
        else:
            self.only_train_lora = config.only_train_lora

        # Optimisers
        # PGD optim initialized in projected_gradient_descent function
        def_lr = self.config.def_config.outer_learning_rate
        self.def_optim = torch.optim.Adam(self.model.parameters(), lr=def_lr)
        if config.def_config.use_cosine_schedule:
            self.schedule = self._make_lr_schedule(base_lr=def_lr, final_lr=def_lr/5, num_epochs=1, iter_per_epoch=config.num_epochs)

        self.model_layers = self.config.model_layers if self.config.model_layers is not None else range(self.model.config.num_hidden_layers)


    def _make_lr_schedule(self, base_lr, final_lr, num_epochs, iter_per_epoch):
        decay_iter = iter_per_epoch * num_epochs
        return final_lr+0.5*(base_lr-final_lr)*(1+np.cos(np.pi*np.arange(decay_iter)/decay_iter)) 
    

    def _load_untargeted_wmdp_data(self) -> tuple[list[str], list[list[str]]]:
        """
        For untargeted data loading and SFT.
        Use wmdp cut utils, but don't return retain_data_list since we load wikitext separately.

        Returns:
            keywords_list: List of keywords for making control vectors in CUT for each dataset.
            forget_data_list: List of lists, where outer list corresponds to each dataset, 
                and inner list is the data for that dataset.
            retain_data_list: List of lists, where outer list corresponds to each dataset,
                and inner list is the data for that dataset.
                For untargeted, this is just wikitext, and it isn't even used.
        """
        retain_corpora = self.config.retain_corpora.split(",")
        forget_corpora = self.config.forget_corpora.split(",")
        keywords_list, forget_data_list, retain_data_list = get_data(
            forget_corpora,
            retain_corpora,
            self.rmu_config.min_len,
            self.rmu_config.max_len,
            self.rmu_config.batch_size,
        )
        return forget_data_list
    

    def _load_sft_dataset(self) -> list[str]:
        """Works for wikitext and alpaca."""
        if self.config.sft_corpora == "wikitext":
                raw_data = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
        elif self.config.sft_corpora == "alpaca":
                raw_data = load_dataset("tatsu-lab/alpaca", "default", split="train")
        data = []
        for x in raw_data:
            if len(x['text']) > self.config.rmu_config.min_len:
                data.append(str(x['text'][:self.config.rmu_config.max_len]))
        return data
    

    def _make_untargeted_wmdp_dataloaders(
        self, 
        data: Union[List[str], List[List[str]]], 
        tokenizer: AutoTokenizer, 
        sft: Optional[bool] = False,
    ) -> Union[DataLoader, tuple[DataLoader, DataLoader]]:
        """
        A very baked in and non generic function.
        Don't use for anything except WMDP exactly as loaded with tuple list [bio, cyber].

        Args:
            sft: Flag, if set True: means that the data list is just multiple copies of the same dataset.
                This is a relic of the get_data function from the CUT codebase, and probably can be refactored.
                This will only be used to generate supervised fine tuning dataloader for SFT portion of R2D2 loss in LAT.
        Returns:
            Dataloaders.
        """
        if sft:
            return DataLoader(
                data, 
                shuffle=True, 
                batch_size=self.config.sft_batch_size, 
                collate_fn=WMDPLATDataCollator(tokenizer, truncate_length=self.config.data_truncate_length), 
                drop_last=True,
            )
            
        wmdp_bio_dataloader = DataLoader(
            data[0], 
            shuffle=True, 
            batch_size=self.config.lat_batch_size, 
            collate_fn=WMDPLATDataCollator(tokenizer, truncate_length=self.config.data_truncate_length), 
            drop_last=True,
        )
        wmdp_cyber_dataloader = DataLoader(
            data[1], 
            shuffle=True, 
            batch_size=self.config.lat_batch_size, 
            collate_fn=WMDPLATDataCollator(tokenizer, truncate_length=self.config.data_truncate_length), 
            drop_last=True,
        )
        return wmdp_bio_dataloader, wmdp_cyber_dataloader


    def _load_targeted_wmdp_data(self) -> tuple[Dataset, Dataset]:
        """
        Use both retain and unlearn WMDP datasets.
        Valid reteain keywords: all wikitext or all bio/cyber retain. Not allowed to mix wikitext and WMDP retain.
        
        Returns:
            keywords_list: List of keywords for making control vectors in CUT for each dataset.
            bio_dataset: Concatenated dataset for bio, containing adv_labels and def_labels columns.
            cyber_dataset: Concatenated dataset for cyber, containing adv_labels and def_labels columns.
        """
        retain_corpora = self.config.retain_corpora.split(",")
        forget_corpora = self.config.forget_corpora.split(",")

        # Load and rename datasets for 'forget' corpora
        # Only bio forget needs to be locally loaded
        hf_location = "cais/wmdp-corpora"
        for d in forget_corpora:
            dataset_path = f"wmdp/data/{d}.jsonl"
            if d == "bio-forget-corpus":
                bio_forget = load_dataset('json', data_files=dataset_path, split='train')
                bio_forget = bio_forget.rename_column('text', 'adv_tokens')
            elif d == "cyber-forget-corpus":
                cyber_forget = load_dataset(hf_location, name=d, split='train')
                cyber_forget = cyber_forget.rename_column('text', 'adv_tokens')

        if retain_corpora == ['bio-retain-corpus', 'cyber-retain-corpus']:
            # Load and rename datasets for 'retain' corpora
            for d in retain_corpora:
                retain_dataset = load_dataset(hf_location, name=d, split='train')
                retain_dataset = retain_dataset.rename_column('text', 'def_tokens')
                if d == "bio-retain-corpus":
                    bio_retain = retain_dataset
                elif d == "cyber-retain-corpus":
                    cyber_retain = retain_dataset
        elif retain_corpora == ['wikitext', 'wikitext']:
            bio_retain = cyber_retain = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
            bio_retain = bio_retain.rename_column('text', 'def_tokens')
            cyber_retain = cyber_retain.rename_column('text', 'def_tokens')

        def merge_rows(example1, example2):
            return {'adv_tokens': example1['adv_tokens'], 'def_tokens': example2['def_tokens']}

        min_bio_length, min_cyber_length = min(len(bio_forget), len(bio_retain)), min(len(cyber_forget), len(cyber_retain))
        bio_dataset = bio_forget.select(range(min_bio_length)).map(
            lambda x, idx: merge_rows(x, bio_retain[idx]),
            with_indices=True,
        )
        cyber_dataset = cyber_forget.select(range(min_cyber_length)).map(
            lambda x, 
            idx: merge_rows(x, cyber_retain[idx]), 
            with_indices=True
        )
        bio_dataset = bio_dataset.remove_columns(['title', 'abstract', 'doi'])

        return bio_dataset, cyber_dataset
    

    def _make_targeted_wmdp_dataloaders(
        self, 
        bio_dataset: Dataset,
        cyber_dataset: Dataset, 
        tokenizer: AutoTokenizer, 
    ) -> Union[DataLoader, tuple[DataLoader, DataLoader]]:
        """
        A very baked in and non generic function.
        Don't use for anything except WMDP exactly as loaded with tuple list [bio, cyber].

        Args:
            sft: Flag, if set True: means that the data list is just multiple copies of the same dataset.
                This is a relic of the get_data function from the CUT codebase, and probably can be refactored.
                This will only be used to generate supervised fine tuning dataloader for SFT portion of R2D2 loss in LAT.
        Returns:
            Dataloaders.
        """
        wmdp_bio_dataloader = DataLoader(
            bio_dataset, 
            shuffle=True, 
            batch_size=self.config.lat_batch_size, 
            collate_fn=WMDPLATTargetedDataCollator(tokenizer, truncate_length=self.config.data_truncate_length), 
            drop_last=True,
        )
        wmdp_cyber_dataloader = DataLoader(
            cyber_dataset, 
            shuffle=True, 
            batch_size=self.config.lat_batch_size, 
            collate_fn=WMDPLATTargetedDataCollator(tokenizer, truncate_length=self.config.data_truncate_length), 
            drop_last=True,
        )
        return wmdp_bio_dataloader, wmdp_cyber_dataloader
    

    # def _evaluate(self, idx) -> dict[str, float]:
    #     """Capabilities and WMDP."""
    #     wmdp_scores = {}
    #     wmdp_datasets = ["chem", "bio", "cyber"]
    #     for dataset_name in wmdp_datasets:
    #         wmdp_mcq = WMDPTask(dataset_name)
    #         wmdp_scores[f"wmdp_{dataset_name}"] = wmdp_mcq.get_accuracy(self.model, self.tokenizer, batch_size=self.config.minibatch_size, n_batches=50) - 0.25
    #     avg_wmdp_score = sum([abs(v) for v in wmdp_scores.values()])/len(wmdp_scores)

    #     # Evaluate everything less frequently
    #     if idx % (10 * self.config.eval_steps) == 0:
    #         capabilities: dict = run_general_evals(self.model, model_type="zephyr")
    #         avg_capabilities = sum(capabilities.values())/len(capabilities)
    #         combined_evals = capabilities | wmdp_scores | {"pareto_score": avg_capabilities / avg_wmdp_score + avg_capabilities}
    #     else:
    #         mmlu = MMLUTask()
    #         mmlu = mmlu.get_accuracy(self.model, tokenizer=self.tokenizer, temperature=0, batch_size=25, n_batches=40, verbose=False)
    #         combined_evals = wmdp_scores | {"MMLU": mmlu} | {"pareto_score": mmlu/ avg_wmdp_score}

    #     return combined_evals

    def _evaluate_phillip(self, model: nn.Module) -> dict:
        """Create right padded tokenizer specific to this evaluation method."""
        tokenizer = AutoTokenizer.from_pretrained(
            "HuggingFaceH4/zephyr-7b-beta", trust_remote_code=True, use_fast=False
        )
        tokenizer.pad_token_id = tokenizer.eos_token_id
        tokenizer.padding_side = "right"
        tokenizer.truncation_side = "right"
        tokenizer.mask_token_id = tokenizer.eos_token_id
        tokenizer.sep_token_id = tokenizer.eos_token_id
        tokenizer.cls_token_id = tokenizer.eos_token_id

        wmdp_scores = {}
        num_iters = 10
        subtasks = ["wmdp-bio", "wmdp-chem", "wmdp-cyber"]
        for subtask in subtasks:
            wmdp = WMDP_MCTask(batch_size=16, tokenizer=tokenizer, subset=subtask)
            unlearned_accuracy = 0
            for _ in tqdm(range(num_iters)):
                unlearned_accuracy += wmdp.get_test_accuracy(model)
            wmdp_scores[subtask] = unlearned_accuracy / num_iters
        avg_wmdp_score = sum([abs(v) for v in wmdp_scores.values()])/len(wmdp_scores)
        print(wmdp_scores)

        # Evaluate everything less frequently
        if self.idx % (10 * self.config.eval_steps) == 0:
            capabilities: dict = run_general_evals(self.model, model_type="zephyr")
            avg_capabilities = sum(capabilities.values())/len(capabilities)
            combined_evals = capabilities | wmdp_scores | {"pareto_score": avg_capabilities / avg_wmdp_score + avg_capabilities}
        else:
            mmlu = MMLUTask()
            mmlu = mmlu.get_accuracy(self.model, tokenizer=self.tokenizer, temperature=0, batch_size=25, n_batches=40, verbose=False)
            combined_evals = wmdp_scores | {"MMLU": mmlu} | {"pareto_score": mmlu/ avg_wmdp_score}

        return combined_evals


    def _evaluate_harness(self, is_sample=True) -> dict[str, float]:
        """
        Evaluate with lm-evaluation-harness.
        Involves saving peft model.

        Format of output from lm_eval.simple_evaluate:
        {'wmdp_bio': {'acc,none': 0.5553809897879026, 'acc_stderr,none': 0.01393304254299126, 'alias': 'wmdp_bio'}}
        """
        self.model.save_pretrained(f"wmdp_models/{self.run_name}")
        model_to_eval = HFLM(
            pretrained=model_dict[self.config.model_name],
            peft=f"wmdp_models/{self.run_name}",
            dtype=torch.bfloat16,
            device="auto"
        )
        if is_sample:
            eval_func = partial(
                lm_eval.simple_evaluate, 
                model=model_to_eval, 
                batch_size=16, 
                limit=16, 
                cache_requests=True
            )
        else:
            eval_func = partial(
                lm_eval.simple_evaluate, 
                model=model_to_eval, 
                batch_size=512, 
                cache_requests=True
            )
        
        # WMDP
        wmdp_scores_raw: dict[str, dict[str, float]] = eval_func(tasks=["wmdp_bio", "wmdp_chem", "wmdp_cyber"])
        wmdp_scores = {k: v["acc,none"] for k, v in wmdp_scores_raw["results"].items()}
        if is_sample:
            wmdp_vars = {f"{k}_var": v["acc_stderr,none"] for k, v in wmdp_scores_raw["results"].items()}
            wandb.log(wmdp_vars, step=self.idx)
        avg_wmdp_score = sum(wmdp_scores.values())/3

        # Capabilities and side effects
        if is_sample:
            mmlu_raw = eval_func(tasks=["mmlu"])
            mmlu_acc = {"mmlu": mmlu_raw["results"]["mmlu"]["acc,none"]}
            mmlu_var = {f"MMLU_var": mmlu_raw["results"]["mmlu"]["acc_stderr,none"]}
            wandb.log(mmlu_var, step=self.idx)
            combined_evals = wmdp_scores | mmlu_acc | {"pareto_score": mmlu_acc["mmlu"] / avg_wmdp_score}
        else:
            capabilities_raw: dict = eval_func(tasks=["mmlu", "sciq", "hellaswag", "piqa", "lambada", "winogrande", "agieval", "mmlu_virology", "mmlu_college_computer_science", "mmlu_high_school_computer_science", "mmlu_college_biology", "mmlu_high_school_biology"])
            capabilities = {k: v["acc,none"] for k, v in capabilities_raw["results"].items()}
            avg_capabilities = sum(capabilities.values())/len(capabilities)
            combined_evals = capabilities | wmdp_scores | {"pareto_score": avg_capabilities / avg_wmdp_score + avg_capabilities}
            
        del model_to_eval
        return combined_evals
    

    def _load_model(self, hf_api_key: str) -> tuple[nn.Module, AutoTokenizer]:
        """Model and tokenizer loading specifically for WMDP benchmark."""
        model = AutoModelForCausalLM.from_pretrained(
            model_dict[self.config.model_name],
            torch_dtype=torch.bfloat16,
            token=hf_api_key,
            trust_remote_code=True,
            device_map="auto",
        )
        tokenizer = AutoTokenizer.from_pretrained(
            model_dict[self.config.model_name],
            token=hf_api_key,
            trust_remote_code=True,
            use_fast=False,
        )

        tokenizer.pad_token_id = tokenizer.eos_token_id
        tokenizer.padding_side = "left"
        tokenizer.mask_token_id = tokenizer.eos_token_id
        tokenizer.sep_token_id = tokenizer.eos_token_id
        tokenizer.cls_token_id = tokenizer.eos_token_id
        
        peft_config = LoraConfig(
            r=self.config.lora_config.lora_rank,
            target_modules=OmegaConf.to_container(self.config.lora_config.lora_target_modules),
        )
        model = get_peft_model(model, peft_config)
        print(f"PEFT model is on {self.device}")
        return model, tokenizer
    

    def _set_logger(self) -> None:
        """WandB as default."""
        logging.info(f"Hydra current working directory: {os.getcwd()}")
        logger_params = {
            # "name": self.run_name, # Had to get rid of this to make it work with sweeps
            "project": self.config.wandb_config.wandb_project_name,
            "settings": wandb.Settings(start_method="thread"),
            "config": OmegaConf.to_container(self.config, resolve=True, throw_on_missing=True),
            "mode": "disabled" if not self.config.wandb_config.log_to_wandb else "online",
            "entity": self.config.wandb_config.entity_name,
        }
        run = wandb.init(**logger_params)
        return run


    def _disable_model_gradients(self) -> None:
        for param in self.model.parameters():
            param.requires_grad_(False)
    

    def _enable_model_gradients(self) -> None:
        n_layers = self.model.config.num_hidden_layers
        for i in range(n_layers):
            if i in self.model_layers:
                if self.only_train_lora:
                    for name, param in self.model.get_submodule(self.config.pgd_config.model_layers_module)[i].named_parameters():
                        if "lora_" in name:
                            param.requires_grad_(True)
                else:
                    self.model.get_submodule(self.config.pgd_config.model_layers_module)[i].requires_grad_(True)
    

    def _train_defence(
        self, 
        batch: dict[str, torch.Tensor], 
        hooks: list[CustomHook], 
        sft_batch: dict[str, torch.Tensor],
        zero_grad: bool,
        do_grad_step: bool,
    ) -> dict[str, float]:
        # Initialize optimizer and loss
        losses = {}
        if zero_grad:
            self.def_optim.zero_grad()

        # Compute the defense        
        do_defense_step(
            model=self.model,
            batch=batch,
            losses_dict=losses,
            wrappers=hooks,
            sft_batch=sft_batch,
            coefs=self.config.def_config.def_loss_coefs,
            log_loss=do_grad_step,
            device=self.device,
            towards_loss_type=self.config.def_config.towards_loss_type,
            away_loss_type=self.config.def_config.away_loss_type,
        )
        zero_nan_grads(self.model)
        # Do gradient step
        if do_grad_step:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.clip_grad)
            self.def_optim.step()

        return losses
    

    def _train_attack(
        self, 
        batch: dict[str, torch.Tensor], 
        do_grad_step: bool,
        pca_kwargs: dict = None,
    ) -> None:
        config = self.config.pgd_config
        return projected_gradient_descent(
            batch=batch,
            model=self.model,
            model_layers_module=config.model_layers_module,
            layer_list=config.pgd_layers,
            epsilon=config.epsilon,
            learning_rate=config.inner_learning_rate,
            pgd_iterations=config.pgd_iterations_per_step,
            loss_coefs=config.adv_loss_coefs,
            log_loss=do_grad_step,
            device=self.device,
            use_cut=self.config.use_cut,
            towards_loss_type=config.towards_loss_type,
            away_loss_type=config.away_loss_type,
            pca_kwargs=pca_kwargs,
        )
    

    def _lat_training_step(
        self, 
        batch: dict[str, torch.Tensor],
        sft_batch: Optional[dict[str, torch.Tensor]] = None,
        pca_kwargs: dict = None,
    ) -> None:
        """Make minibatches and train with gradient accumulation, or train batches directly."""
        # Gradient accumulation
        if not self.config.minibatch_size == self.config.lat_batch_size:
            minibatch_steps = range(0, self.config.lat_batch_size, self.config.minibatch_size)
            adv_hooks = []

            # Train adversary with minibatch accumulation
            for start_idx in minibatch_steps:
                curr_batch = get_minibatch(batch, start_idx, self.config.minibatch_size)

                self._disable_model_gradients()
                losses, hooks, _ = self._train_attack(
                    batch=curr_batch,
                    do_grad_step=start_idx==minibatch_steps[-1],
                    pca_kwargs=pca_kwargs,
                )
                adv_hooks.append(hooks)
                # Disable adversary for this minibatch
                for hook in hooks:
                    hook.enabled = False
            # Log last value to WandB
            if self.post_adv_callback is not None:
                self.post_adv_callback(result=losses, epoch=self.idx)
            
            # Train defence with minibatch accumulation
            for _ in range(self.config.def_config.model_iterations_per_step):
                for i, start_idx in enumerate(minibatch_steps):
                    curr_batch = get_minibatch(batch, start_idx, self.config.minibatch_size)
                    curr_sft_batch = get_minibatch(sft_batch, start_idx, self.config.minibatch_size) if self.config.def_config.def_loss_coefs.sft > 0 else None
                    hooks = adv_hooks[i]
                    # Enable adversary for this minibatch
                    for hook in hooks:
                        hook.enabled = True
                    self._enable_model_gradients()
                    def_losses = self._train_defence(
                        batch=curr_batch,
                        hooks=hooks,
                        sft_batch=curr_sft_batch,
                        zero_grad=start_idx==0,
                        do_grad_step=start_idx==minibatch_steps[-1],
                    )
                    for hook in hooks:
                        hook.enabled = False
        
        # No gradient accumulation
        else:
            # Train adversary
            self._disable_model_gradients()
            losses, hooks, _ = self._train_attack(
                batch=batch,
                do_grad_step=True,
            )
            if self.post_adv_callback is not None:
                self.post_adv_callback(result=losses, epoch=self.idx)

            # Train defence
            self._enable_model_gradients()
            for _ in range(self.config.def_config.model_iterations_per_step):
                def_losses = self._train_defence(
                    batch=batch,
                    hooks=hooks,
                    sft_batch=sft_batch,
                    zero_grad=True,
                    do_grad_step=True,
                )
        
        # Log results
        losses.update(def_losses)
        if self.post_def_callback is not None:
            self.post_def_callback(result=def_losses, epoch=self.idx)
        clear_hooks(self.model)


    def train(self) -> None:
        """
        Main training loop for WMDP benchmark.
        Set logger and call callback functions.
        Iterate over num_epochs (which really is just PGD iterations), or until wallclock runs out.
            Will always default to infinite training unless wallclock time not specified.
        Check if using minibatch training.

        An epoch here equates to seeing a single batch of data.
        """
        # Call initialisation callbacks
        if self.init_callback is not None:
            self.init_callback(result={}, epoch=-1)

        start_time = time.time()
        self.idx = 0
        epochs_iter = tqdm(range(self.config.num_epochs)) if self.config.num_epochs is not None else tqdm(itertools.count())
        print(f"Config epochs {self.config.num_epochs}")

        for epoch_idx in tqdm(epochs_iter):

            if self.config.def_config.reinitialize_def_optim:
                self.def_optim = torch.optim.AdamW(
                    self.model.parameters(),
                    lr=self.schedule[self.idx] if self.config.def_config.use_cosine_schedule else self.config.def_config.outer_learning_rate,
                )
            # batch = next(self.cyber_dataloader) if self.idx % 5 else next(self.bio_dataloader)
            batch = next(self.bio_dataloader) # Temporary: USE BIO FOR NOW
            sft_batch = next(self.sft_dataloader) if self.sft_dataloader else None

            # try:
            self._lat_training_step(batch, sft_batch)
        
            # Log evaluation metrics every 10 its
            if self.idx % self.config.eval_steps == 0:
                with torch.inference_mode():
                    eval_accs = self._evaluate_harness()
                eval_and_log(eval_accs, self.idx)

            if self.idx % 20 == 0:
                self._save_model()
            self.idx += 1

            # except Exception as e:
            #     print(f"Error at epoch {epoch_idx} of {self.run_name}: {e}")
            #     os.makedirs("logs", exist_ok=True)
            #     with open(f"logs/{self.run_name}_errors.txt", "a") as f:
            #         f.write(f"Error at epoch {epoch_idx} of {self.run_name}: {e}\n")
            #     break

            elapsed_time = time.time() - start_time
            if self.config.time_limit is not None and elapsed_time > self.config.time_limit:
                print(f"Reached {elapsed_time} seconds at epoch {epoch_idx}")
                break
        
        self._save_model()
        self._evaluate_harness(is_sample=False)
        for attr in list(self.__dict__.keys()):
            del attr
        torch.cuda.empty_cache()
        wandb.finish()
    

    def _save_model(self):
        path = f"wmdp_models/{self.run_name}"
        clear_hooks(self.model)
        self.model.save_pretrained(path)
        self.tokenizer.save_pretrained(path)
        print(f"Saved model to {path}")


class RunLATCUT(RunLAT):
    def __init__(
        self, 
        config: OmegaConf, 
        cuda_id: int,
        sweep_params: Optional[list[str]],
        init_callback: Optional[Callable] = None,
        post_def_callback: Optional[Callable] = None,
        post_adv_callback: Optional[Callable] = None,
    ):
        """
        Args:
            cuda_id: Each Run instance associated with a single GPU. Allows sweeps to call multiple Run instances on multiple GPUs.
            sweep_params: Any parameters that are changed by the sweep.
            init_callback: Typically, an evaluation function.
            post_def_callback: "".
            post_adv_callback: "".
        """
        assert config.use_cut, "This is a CUT only script."
        assert isinstance(config.pgd_config.pgd_layers, int), "pgd_layers must be an integer for CUT as lists are not yet supported"

        if torch.cuda.is_available():
            self.device = torch.device(f"cuda:{cuda_id}")
        elif torch.backends.mps.is_available():
            self.device = torch.device("mps")
        else:
            self.device = torch.device("cpu")
        
        self.config = config

        self.def_coefs = config.def_config.def_loss_coefs

        if config.only_train_lora is None:
            self.only_train_lora = isinstance(self.model, PeftModel)
        else:
            self.only_train_lora = config.only_train_lora
        
        # Callbacks
        self.init_callback = init_callback
        self.post_adv_callback = post_adv_callback
        self.post_def_callback = post_def_callback

        # Model
        self.model, self.tokenizer = self._load_model(hf_api_key, use_peft=config.use_peft, frozen_model=False)
        self.frozen_model, _ = self._load_model(hf_api_key, frozen_model=True)

        if config.replicate_cut:
            self.forget_data_list, self.retain_data_list = get_data(
                config.rmu_config.forget_corpora,
                config.rmu_config.retain_corpora,
                config.rmu_config.min_len,
                config.rmu_config.max_len,
                config.rmu_config.batch_size,
            )
            # TODO: uncommented below block
            bio_dataset, cyber_dataset = self._load_targeted_wmdp_data()
            self.bio_dataloader, self.cyber_dataloader = self._make_targeted_wmdp_dataloaders(bio_dataset, cyber_dataset, self.tokenizer)
            self.bio_dataloader, self.cyber_dataloader = itertools.cycle(self.bio_dataloader), itertools.cycle(self.cyber_dataloader)
            self.model_layers = self.config.model_layers if self.config.model_layers is not None else range(self.model.config.num_hidden_layers)

            self.run_rmu_copy(args=config.rmu_config)

        else:
            if config.use_untargeted: # Untargeted LAT, use unlearn for defence and adversary
                forget_list = self._load_untargeted_wmdp_data()
                self.bio_dataloader, self.cyber_dataloader = self._make_untargeted_wmdp_dataloaders(forget_list, self.tokenizer, sft=False)
            else: # Use W           MDP retain and unlearn - note logic is *totally different* to above
                assert config.forget_corpora == 'bio-forget-corpus,cyber-forget-corpus'
                bio_dataset, cyber_dataset = self._load_targeted_wmdp_data()
                self.bio_dataloader, self.cyber_dataloader = self._make_targeted_wmdp_dataloaders(bio_dataset, cyber_dataset, self.tokenizer)
            self.bio_dataloader, self.cyber_dataloader = itertools.cycle(self.bio_dataloader), itertools.cycle(self.cyber_dataloader)
            # SFT dataloader
            if config.def_config.def_loss_coefs.sft > 0:
                sft_dataset: list[str] = self._load_sft_dataset()
                self.sft_dataloader = self._make_untargeted_wmdp_dataloaders(sft_dataset, self.tokenizer, sft=True)
                self.sft_dataloader = itertools.cycle(self.sft_dataloader)
            else:
                assert config.def_config.def_loss_coefs["sft"] == 0
                self.sft_dataloader = None
            assert next(self.bio_dataloader)["def_tokens"].shape[0] == self.config.lat_batch_size, "Batch size mismatch"

            # Put optimizer for defence model here 
            # PGD optim initialized in projected_gradient_descent function
            self.def_optim = torch.optim.Adam(
                get_params(self.model, layer_ids=OmegaConf.to_container(self.config.rmu_config.layer_ids), param_ids=self.config.rmu_config.param_ids), 
                lr=config.def_config.outer_learning_rate
            )
            # self.def_optim = torch.optim.Adam(self.model.parameters(), lr=config.def_config.outer_learning_rate)

            run = self._set_logger()
            if sweep_params: # For wandb sweeps: update with wandb values
                config = update_with_wandb_config(config, sweep_params)
            self.run_name = f"{config.model_name.lower()}_layers{config.pgd_config.pgd_layers}_alpha{config.rmu_config.alpha}_ccoef{config.rmu_config.steering_coef}_{config.wandb_config.additional_run_name}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
            run.name = self.run_name

            self.train_cut()


    def _load_untargeted_wmdp_data(self) -> tuple[list[str], list[list[str]]]:
        """
        For untargeted data loading and SFT.
        Use wmdp cut utils, but don't return retain_data_list since we load wikitext separately.

        Returns:
            keywords_list: List of keywords for making control vectors in CUT for each dataset.
            forget_data_list: List of lists, where outer list corresponds to each dataset, 
                and inner list is the data for that dataset.
            retain_data_list: List of lists, where outer list corresponds to each dataset,
                and inner list is the data for that dataset.
                For untargeted, this is just wikitext, and it isn't even used.
        """
        retain_corpora = self.config.retain_corpora.split(",")
        forget_corpora = self.config.forget_corpora.split(",")
        keywords_list, forget_data_list, retain_data_list = get_data(
            forget_corpora,
            retain_corpora,
            self.config.rmu_config.min_len,
            self.config.rmu_config.max_len,
            self.config.rmu_config.batch_size,
        )
        return forget_data_list, retain_data_list
    

    def _make_rmu_vec(self, batch, module):
        """
        Make uniform random drawn from 0-1 attack vector.
        Call once at start of training loop.
        """
        activations = forward_with_cache(
            self.frozen_model, batch["def_tokens"].to(self.device), module=module, no_grad=True
        ).to(self.device)
        random_vector = torch.rand(activations.shape, device=self.device)
        self.rmu_vec = self._norm_attacks(random_vector)

        # # Allow size (1,1,hidden) to broadcast later
        # random_vector = torch.rand(1, 1, self.model.config.hidden_size, device=self.device)
        # self.rmu_vec = random_vector / torch.norm(random_vector)


    def _load_model(self, hf_api_key: str, use_peft: Optional[bool] = False, frozen_model: Optional[bool] = False):
        """Model and tokenizer loading specifically for WMDP benchmark.
        
        Key for saving compute is PEFT config can be loaded and unloaded.
        """
        model = AutoModelForCausalLM.from_pretrained(
            model_dict[self.config.model_name],
            torch_dtype=torch.bfloat16,
            token=hf_api_key,
            trust_remote_code=True,
            device_map="auto",
        ).to(self.device)
        tokenizer = AutoTokenizer.from_pretrained(
            model_dict[self.config.model_name],
            token=hf_api_key,
            trust_remote_code=True,
            use_fast=False,
        )

        if not frozen_model:
            tokenizer.pad_token_id = tokenizer.eos_token_id
            tokenizer.padding_side = "right"
            tokenizer.mask_token_id = tokenizer.eos_token_id
            tokenizer.sep_token_id = tokenizer.eos_token_id
            tokenizer.cls_token_id = tokenizer.eos_token_id
        
            if use_peft:
                peft_config = LoraConfig(
                    r=self.config.lora_config.lora_rank,
                    target_modules=OmegaConf.to_container(self.config.lora_config.lora_target_modules),
                )
                model = get_peft_model(model, peft_config)
                print(f"PEFT model is on {self.device}")
            else: # Don't use all layers, only some layers full finetune
                self.model = model
                print(f"Model without PEFT is on {self.device}")
        
        return model, tokenizer


    def _norm_attacks(self, attacks: torch.Tensor) -> torch.Tensor:
        """
        Normalize each 'attack' tensor such that attacks[i][j] has norm 1 (per batch example).
        """
        normalized_attacks = attacks / attacks.norm(dim=-1, keepdim=True)
        return normalized_attacks


    # TODO: make adversary possible on multiple layers (matching multiple layers)
    # Currently adversary attacks is a list to support multiple layers but we only use the first
    def _train_defence(
        self, 
        batch: dict[str, torch.Tensor], 
        do_grad_step: bool,
        updated_module: nn.Module,
        frozen_module: nn.Module,
        zero_grad: bool,
        log_loss: Optional[bool] = True,
        sft_batch: Optional[dict[str, torch.Tensor]] = None, 
        attacks: Optional[List[torch.Tensor]] = [],
        hooks: Optional[list[CustomHook]] = [], 
    ) -> dict[str, float]:
        """
        Defence training always with adversary perturbations active in this LAT version of RMU.

        Modifications made compared to non-CUT version:
        - Assume always use toward and away data, but this time to construct the relevant CUT related activation losses.

        Modifications to CUT:
        - Attacks from adversary added implicitly, and defence now has to match activations, so we don't necessarily need a forget loss.
        - SFT batch now optional, since CUT doesn't automatically use it.
        """
        losses = {"total": 0}
        if zero_grad:
            self.def_optim.zero_grad()

        # Retain loss
        model_towards_tokens = batch["def_tokens"] if "def_tokens" in batch else batch["tokens"]
        model_towards_tokens = model_towards_tokens[batch["def_labels_mask"]].to(self.device)
        updated_activations = forward_with_cache(
            self.model, model_towards_tokens, module=updated_module, no_grad=False,
        ).to(self.device)
        frozen_activations = forward_with_cache(
            self.frozen_model, model_towards_tokens, module=frozen_module, no_grad=True
        ).to(self.device)
        retain_loss = torch.nn.functional.mse_loss(
            updated_activations, frozen_activations
        )
        retain_loss *= self.config.rmu_config.alpha[0]
        (self.def_coefs["toward"] * retain_loss).backward()
        losses["retain"] = retain_loss.item()
        losses["total"] += retain_loss.item()

        # Forget loss
        model_away_tokens = batch["adv_tokens"] if "adv_tokens" in batch else batch["tokens"]
        model_away_tokens = model_away_tokens[batch["adv_labels_mask"]].to(self.device)
        updated_activations = forward_with_cache(
            self.model, model_away_tokens, module=updated_module, no_grad=False
        ).to(self.device)
        random_vector = torch.rand(updated_activations.shape, device=self.device)
        self.rmu_vec = self._norm_attacks(random_vector)
        control_vec = (self.config.rmu_config.steering_coef * self.rmu_vec).to(self.device)
        unlearn_loss = torch.nn.functional.mse_loss(
            updated_activations, control_vec
        )
        (self.def_coefs["away"] * unlearn_loss).backward()
        losses["unlearn"] = unlearn_loss.item()
        losses["total"] += unlearn_loss.item()

        # if self.def_coefs["sft"] > 0:
        #     sft_tokens = sft_batch["def_tokens"].to(self.device) if "def_tokens" in sft_batch else sft_batch["tokens"].to(self.device)
        #     sft_labels_mask = sft_batch["def_labels_mask"].to(self.device) if "def_labels_mask" in batch else torch.ones_like(batch["def_labels"]).to(self.device)

        #     for wrapper in hooks:
        #         wrapper.enabled = False

        #     with torch.autocast(device_type="cuda"):
        #         logits = self.model(sft_tokens.to(self.device)).logits
        #         final_logits = logits[:, :-1][sft_labels_mask[:, 1:]]
        #         flattened_sft_labels = sft_tokens[:, 1:][sft_labels_mask[:, 1:]]
        #         sft_loss = nn.functional.cross_entropy(final_logits, flattened_sft_labels)

        #     (self.def_coefs["sft"] * sft_loss).backward()
        #     losses["sft"] = sft_loss.item()
        #     losses["total"] += sft_loss.item()
    
        #     for wrapper in hooks:
        #         wrapper.enabled = True
            
        zero_nan_grads(self.model)
        # Do gradient step
        if do_grad_step:
            # torch.nn.utils.clip_grad_norm_(
            #     self.model.parameters(), self.config.clip_grad)
            self.def_optim.step()
        
        # This will actually only return loss from last loop iteration
        if log_loss:
            losses_dict = {}
            for key in losses:
                losses_dict["def_"+key] = losses[key]
            
        return losses_dict
    

    def _train_attack(
        self, 
        batch: dict[str, torch.Tensor], 
        do_grad_step: bool,
        pca_kwargs: Optional[dict] = None,
    ) -> None:
        config = self.config.pgd_config
        return projected_gradient_descent(
            batch=batch,
            model=self.model,
            model_layers_module=config.model_layers_module,
            layer_list=config.pgd_layers,
            epsilon=config.epsilon,
            learning_rate=config.inner_learning_rate,
            pgd_iterations=config.pgd_iterations_per_step,
            loss_coefs=config.adv_loss_coefs,
            log_loss=do_grad_step,
            device=self.device,
            use_cut=self.config.use_cut,
            towards_loss_type=config.towards_loss_type,
            away_loss_type=config.away_loss_type,
            pca_kwargs=pca_kwargs,
        )


    def _lat_training_step(
        self, 
        batch: dict[str, torch.Tensor],
        frozen_module: nn.Module,
        updated_module: nn.Module,
        sft_batch: Optional[dict[str, torch.Tensor]] = None,
    ) -> None:
        """Batch comes directly from batch in dataloader, or from minibatch."""
        assert isinstance(batch, dict), "Batch is not dict!"

        if not self.config.minibatch_size == self.config.lat_batch_size:
            minibatch_steps = range(0, self.config.lat_batch_size, self.config.minibatch_size)
            # adv_hooks = []
            # adv_attacks = []

            # # Train adversary with minibatch accumulation
            # for start_idx in minibatch_steps:
            #     if start_idx + self.config.minibatch_size > self.config.sft_batch_size: 
            #         break
            #     curr_batch = get_minibatch(batch, start_idx, self.config.minibatch_size)

            #     self._disable_model_gradients()
            #     losses, hooks, _ = self._train_attack(
            #         batch=curr_batch,
            #         do_grad_step=start_idx==minibatch_steps[-1],
            #     )
            #     adv_hooks.append(hooks)

            #     # Used for old version of CUT + LAT
            #     # adv_attacks.append(attacks)

            #     # Log to WandB
            #     if self.post_adv_callback is not None:
            #         self.post_adv_callback(result=losses, epoch=self.idx)
            #     # Disable adversary for this minibatch
            #     for hook in hooks:
            #         hook.enabled = False
            
            # Train defence with minibatch accumulation
            for _ in range(self.config.def_config.model_iterations_per_step):
                for i, start_idx in enumerate(minibatch_steps):
                    curr_batch = get_minibatch(batch, start_idx, self.config.minibatch_size)
                    curr_sft_batch = get_minibatch(sft_batch, start_idx, self.config.minibatch_size) if self.config.def_config.def_loss_coefs.sft > 0 else None

                    # Get RMU steering vector if at start of training
                    if self.idx == 0:
                        self._make_rmu_vec(curr_batch, updated_module)

                    # hooks = adv_hooks[i]
                    # Enable adversary for this minibatch
                    # for hook in hooks:
                    #     hook.enabled = True
                    self._enable_model_gradients()
                    def_losses = self._train_defence(
                        batch=curr_batch,
                        # hooks=hooks,
                        # attacks=adv_attacks[i],
                        sft_batch=curr_sft_batch,
                        frozen_module=frozen_module,
                        updated_module=updated_module,
                        zero_grad=start_idx==0,
                        do_grad_step=start_idx==minibatch_steps[-1],
                    )
                    # for hook in hooks:
                    #     hook.enabled = False
        
        else:
            curr_batch, curr_sft_batch = batch, sft_batch

            # Train adversary
            # self._disable_model_gradients()
            # losses, hooks, attacks = self._train_attack(
            #     batch=curr_batch,
            #     do_grad_step=True,
            # )
            # if self.post_adv_callback is not None:
            #     self.post_adv_callback(result=losses, epoch=self.idx)

            # Train defence
            # self._enable_model_gradients()
            for _ in range(self.config.def_config.model_iterations_per_step):
                def_losses = self._train_defence(
                    batch=curr_batch,
                    # hooks=hooks,
                    # attack=attacks,
                    sft_batch=curr_sft_batch,
                    frozen_module=frozen_module,
                    updated_module=updated_module,
                    zero_grad=True,
                    do_grad_step=True,
                )
        
        # Log results
        # losses.update(def_losses)
        if self.post_def_callback is not None:
            self.post_def_callback(result=def_losses, epoch=self.idx)
        clear_hooks(self.model)


    def train_cut(self) -> None:
        if self.init_callback is not None:
            self.init_callback(result={}, epoch=-1)

        start_time = time.time()
        self.idx = 0
        epochs_iter = tqdm(range(self.config.num_epochs)) if self.config.num_epochs is not None else tqdm(itertools.count())
        print(f"Config epochs {self.config.num_epochs}")
        
        if self.config.use_peft:
            module_str="{model_name}.base_model.model.model.layers[{layer_id}]"
        else:
            module_str="{model_name}.model.layers[{layer_id}]"
        updated_module = eval(
            module_str.format(model_name="self.model", layer_id=self.config.pgd_config.pgd_layers+self.config.rmu_config.post_pgd_offset)
        )
        frozen_module = eval(
            module_str.format(model_name="self.frozen_model", layer_id=self.config.pgd_config.pgd_layers)
        )

        for epoch_idx in tqdm(epochs_iter):
            """
            This doesn't need refactoring to use the setup,
            but pgd_layers must be exactly an integer. 
            """
            if self.config.def_config.reinitialize_def_optim:
                self.def_optim = torch.optim.AdamW(
                    self.model.parameters(),
                    lr=self.config.def_config.outer_learning_rate
                )
            batch = next(self.cyber_dataloader) if self.idx % 2 else next(self.bio_dataloader)
            sft_batch = next(self.sft_dataloader) if self.sft_dataloader else None
            self._lat_training_step(
                batch, 
                sft_batch=sft_batch, 
                frozen_module=frozen_module, 
                updated_module=updated_module
            )
            
            # Log evaluation metrics every 10 its
            if self.idx % self.config.eval_steps == 0:
                with torch.inference_mode():
                    eval_accs = self._evaluate_phillip(self.model)
                eval_and_log(eval_accs, self.idx)

            self.idx += 1

            elapsed_time = time.time() - start_time
            if self.config.time_limit is not None and elapsed_time > self.config.time_limit:
                print(f"Reached {elapsed_time} seconds at epoch {epoch_idx}")
                break
        
        # Evaluate using harness at end
        self._evaluate_harness()

        wandb.finish()
    

    def _save_model(self):
        self.model.save_pretrained(save_directory=f"wmdp_cutlat_models/{self.run_name}")


    def run_rmu(self, args) -> None:  
        """
        Method directly copied from CAIS RMU unlearn code.
        Does not use SFT or PEFT.
        Only trains 3 layers before layer 7.
        Forget - WMDP unlearn corpora; retain - wikitext.

        Args:
            args: OmegaConf dictionary-like object with dot access.
        """
        logger_params = {
            # "name": self.run_name, # Had to get rid of this to make it work with sweeps
            "project": "rmu_replicate",
            "settings": wandb.Settings(start_method="thread"),
            "config": OmegaConf.to_container(args),
            "mode": "online"
        }
        wandb.init(**logger_params, entity="quirky_lats_at_mats")
        
        rmu_config = vars(args)
        print("====rmu Config====")
        print("\n".join(f"{k}={v}" for k,v in rmu_config.items()))
        print("=====")

        self.model = self.model.train()
        params = get_params(self.model, args.layer_ids, args.param_ids)
        optimizer = torch.optim.AdamW(params, lr=args.lr)
        frozen_module = eval(
            args.module_str.format(model_name="self.frozen_model", layer_id=args.layer_id)
        )
        updated_module = eval(
            args.module_str.format(model_name="self.model", layer_id=args.layer_id)
        )

        control_vectors_list = []
        for i in range(len(self.forget_data_list)):
            random_vector = torch.rand(1,1, self.model.config.hidden_size, device=self.device)
            control_vec = random_vector / torch.norm(random_vector) * args.steering_coef
            control_vectors_list.append(control_vec)

        num_batches = min(
            args.max_num_batches,
            min([len(f) for f in self.forget_data_list]),
            min([len(r) for r in self.retain_data_list]),
        )
        
        truncation_side = self.tokenizer.truncation_side
        self.tokenizer.truncation_side="right"

        for epoch in range(1):
            print(f"======= Epoch {epoch} =======")
            with tqdm(total=num_batches) as pbar:
                for idx in range(num_batches):
                    self.idx = idx
                    if idx % self.config.rmu_config.eval_freq == 0:
                        with torch.inference_mode():
                            accs = self._evaluate_phillip(self.model)
                        eval_and_log(accs, idx)
                    
                    topic_idx = idx % len(self.forget_data_list)
                    # if not topic_idx:
                    #     pgd_batch = next(self.bio_dataloader)
                    # else:
                    #     pgd_batch = next(self.cyber_dataloader)
                    batch_idx = idx // len(self.forget_data_list)
                    control_vec = control_vectors_list[topic_idx]
                    unlearn_batch = self.forget_data_list[topic_idx][batch_idx]
                    retain_batch = self.retain_data_list[topic_idx][batch_idx]

                    # # ====== Train adversary PGD ===========
                    # self._disable_model_gradients()
                    # losses, _, _ = self._train_attack(
                    #     batch=pgd_batch,
                    #     do_grad_step=True,
                    # )
                    # if self.post_adv_callback is not None:
                    #     self.post_adv_callback(result=losses, epoch=self.idx)
                    # self._enable_model_gradients()

                    # ===== Train RMU ============
                    # Unlearning loss
                    max_length = 512 if topic_idx == 0 else 768
                    # max_length = self.config.data_truncate_length
                    unlearn_inputs = self.tokenizer(
                        unlearn_batch, return_tensors="pt", padding=True, truncation=True, max_length=max_length
                    ).to(self.device)
                    updated_forget_activations = forward_with_cache(
                        self.model, unlearn_inputs, module=updated_module, no_grad=False
                    ).to(self.device)

                    unlearn_loss = torch.nn.functional.mse_loss(
                        updated_forget_activations, control_vec
                    )

                    # Retain loss
                    retain_inputs = self.tokenizer(
                        retain_batch, return_tensors="pt", padding=True, truncation=True, 
                        max_length=512,
                    ).to(self.device)
                    updated_retain_activations = forward_with_cache(
                        self.model, retain_inputs, module=updated_module, no_grad=False
                    ).to(self.device)
                    frozen_retain_activations = forward_with_cache(
                        self.frozen_model, retain_inputs, module=frozen_module, no_grad=True
                    ).to(self.device)

                    retain_loss = torch.nn.functional.mse_loss(
                        updated_retain_activations, frozen_retain_activations
                    )
                    retain_loss *= args.alpha[topic_idx]

                    # Update model
                    loss = unlearn_loss + retain_loss
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    loss_dict = {"total_loss": loss.item(), "unlearn_loss": unlearn_loss.item(), "retain_loss": retain_loss.item()}
                    eval_and_log(loss_dict, idx)

                    # ======= Logging ======
                    if args.verbose:
                        frozen_forget_activations = forward_with_cache(self.frozen_model, unlearn_inputs, module=frozen_module, no_grad=True).to(self.device)
                        unlearn_cosine= torch.nn.functional.cosine_similarity(updated_forget_activations, frozen_forget_activations, dim=-1).mean()
                        retain_cosine = torch.nn.functional.cosine_similarity(updated_retain_activations, frozen_retain_activations, dim=-1).mean()
                        
                        print(f"unlearn_cosine_sim={unlearn_cosine.item()}")
                        print(f"retain_cosine_sim={retain_cosine.item()}")
                        print(f"Topic {topic_idx} updated_forget_activations.norm=",torch.mean(updated_forget_activations.norm(dim=-1).mean(dim=1), dim=0).item())
                        print(f"Topic {topic_idx} frozen_forget_activations.norm=",torch.mean(frozen_forget_activations.norm(dim=-1).mean(dim=1), dim=0).item())
                        print(f"Topic {topic_idx} updated_retain_activations.norm=",torch.mean(updated_retain_activations.norm(dim=-1).mean(dim=1), dim=0).item())
                        print(f"Topic {topic_idx} frozen_retain_activations.norm=",torch.mean(frozen_retain_activations.norm(dim=-1).mean(dim=1), dim=0).item())

                    # clear_hooks(self.model)

                    pbar.update(1)

        self.tokenizer.truncation_side = truncation_side
        # Save model
        if args.output_dir:
            path = args.output_dir
        else:
            date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
            path = f"models/{args.model_name_or_path}_alpha-{args.alpha}_batches-{num_batches}_layer-{args.layer_id}_{date}"
        self.model.save_pretrained(path)
        self.tokenizer.save_pretrained(path)
        print(f"Saved model to {path}")

        wandb.finish()


def eval_and_log(result: dict[str, float], epoch) -> None:
    """Used as callback function in training, so needs to be global and passed into class."""
    wandb.log(result, step=epoch)


def update_with_wandb_config(config: OmegaConf, sweep_params: list[str]) -> OmegaConf:
    """Check if each parameter exists in wandb.config and update it if it does."""
    for param in sweep_params:
        if param in wandb.config:
            print("Updating param with value from wandb config: ", param, wandb.config[param])
            OmegaConf.update(config, param, wandb.config[param], merge=True)
    return config


def load_config(file_path: str) -> dict:
    with open(file_path, 'r') as stream:
        config = yaml.safe_load(stream)
    return config


def process_target(sweep_id, config, sweep_params, thread_id):
    wandb.agent(
        sweep_id, 
        function=lambda: main(config, thread_id, sweep_params),
        count=config.get("sweep_num")
    )


def main(
    config: dict, 
    thread_id: Optional[int] = 0,
    sweep_params: Optional[list[str]] = None,
):
    """GPU number given by thread ID.
    If not sweep, take thread ID 0 (CUDA 0).
    """
    if config["use_cut"]:
        print("Running LAT with CUT")
        lat = RunLATCUT(
            OmegaConf.create(config),
            cuda_id=thread_id,
            sweep_params=sweep_params,
            init_callback=None,
            post_adv_callback=eval_and_log,
            post_def_callback=eval_and_log,
        )
    else:
        print("Running base LAT")
        lat = RunLAT(
            OmegaConf.create(config),
            cuda_id=thread_id,
            sweep_params=sweep_params,
            init_callback=None,
            post_adv_callback=eval_and_log,
            post_def_callback=eval_and_log,
        )
        lat.train()


def process_target(sweep_id, config, sweep_params, thread_id):
    wandb.agent(
        sweep_id, 
        function=lambda: main(config, thread_id, sweep_params),
        count=config.get("wandb_config").get("sweep_num")
    )


if __name__ == "__main__":
    config: dict = load_config("wmdp_lat_main_config.yaml")

    if config["wandb_config"]["sweep"] and config["wandb_config"]["log_to_wandb"]:
        num_processes = torch.cuda.device_count()

        sweep_dict = {
            "program": "wmdp_run_sweep.py",
            "method": "random",
            "metric": {
                "name": "pareto_score",
                "goal": "maximize"
            },
            "name": "WMDP Sweep",
            "entity": "quirky_lats_at_mats",
            "project": "wmdp_lat",
            "parameters": {
                "pgd_config.pgd_layers": {
                    "values": 
                        list(range(5, 21))
                },
                "pgd_config.inner_learning_rate": {
                    "distribution": "uniform",
                    "min": 1e-2,
                    "max": 1e-1,
                },
                "def_config.outer_learning_rate": {
                    "distribution": "uniform",
                    "min": 6e-5,
                    "max": 1e-4,
                },
                "epsilon": {
                    "distribution": "uniform",
                    "min": 1,
                    "max": 3,
                },
                "pgd_config.pgd_iterations_per_step": {
                    "values": list(range(16, 26))
                },
                "def_config.def_loss_coefs.sft": {
                    "distribution": "uniform",
                    "min": 3,
                    "max": 6,
                },
            },
        }
        sweep_params = list(sweep_dict["parameters"].keys())
        main(config=config, sweep_params=sweep_params)
        # prev_sweep_id = config.get("wandb_config").get("sweep_id") 
        # sweep_id = prev_sweep_id if prev_sweep_id else \
        #     wandb.sweep(
        #         sweep=sweep_dict,
        #         entity=config.get("wandb_config").get("entity_name"),
        #         project=config.get("wandb_config").get("wandb_project_name")
        #     )
        # sweep_params = list(sweep_dict["parameters"].keys())

        # processes = [
        #     multiprocessing.Process(
        #         target=process_target,
        #         args=(sweep_id, config, sweep_params, thread_id),
        #     ) for thread_id in range(num_processes)
        # ]

        # print(f"Using {num_processes} processes for {len(sweep_params)} hyperparameter settings")

        # for process in processes:
        #     process.start()
        # for process in processes:
        #     process.join()
    else:
        main(config)

ModuleNotFoundError: No module named 'torch'